# MQTT Subscribe 應用程式

使用 paho-mqtt 套件訂閱並接收 MQTT Broker 的訊息

## 功能說明
- 連接到 MQTT Broker
- 訂閱指定主題
- 接收並顯示訊息
- 處理 JSON 格式訊息


In [11]:
# 導入必要的套件
import sys
# 添加系統套件路徑（如果套件安裝在系統目錄）
sys.path.insert(0, '/usr/lib/python3/dist-packages')

import paho.mqtt.client as mqtt
import time
import json

# MQTT 設定
MQTT_BROKER = "localhost"  # MQTT Broker 地址
MQTT_PORT = 1883           # MQTT 連接埠（默認是 1883）
MQTT_TOPIC = "客廳/溫度"   # 訂閱的主題
MQTT_CLIENT_ID = "subscriber_001"  # 客戶端 ID

print("✓ 套件導入完成")
print(f"✓ MQTT 設定：Broker={MQTT_BROKER}:{MQTT_PORT}, Topic={MQTT_TOPIC}")


✓ 套件導入完成
✓ MQTT 設定：Broker=localhost:1883, Topic=客廳/溫度


## 定義回調函數


In [12]:
# 連接回調函數
def on_connect(client, userdata, flags, rc):
    """
    當客戶端連接到 Broker 時的回調函數
    
    rc: 連接結果碼
    0: 連接成功
    1: 連接失敗 - 協議版本不正確
    2: 連接失敗 - 客戶端 ID 無效
    3: 連接失敗 - 伺服器無法使用
    4: 連接失敗 - 用戶名或密碼錯誤
    5: 連接失敗 - 未授權
    """
    if rc == 0:
        print(f"✓ 成功連接到 MQTT Broker: {MQTT_BROKER}:{MQTT_PORT}")
        # 連接成功後立即訂閱主題
        print(f"✓ 訂閱主題: {MQTT_TOPIC}")
        client.subscribe(MQTT_TOPIC, qos=1)
    else:
        print(f"✗ 連接失敗，錯誤碼: {rc}")

# 訂閱回調函數
def on_subscribe(client, userdata, mid, granted_qos):
    """
    當訂閱成功時的回調函數
    
    mid: 訂閱請求的訊息 ID
    granted_qos: 被授予的 QoS 等級列表
    """
    print(f"✓ 訂閱成功 (訊息 ID: {mid}, QoS: {granted_qos[0]})")
    print("等待接收訊息...\n")

# 接收訊息回調函數
def on_message(client, userdata, msg):
    """
    當收到訊息時的回調函數
    
    msg: 訊息物件
    - msg.topic: 主題名稱
    - msg.payload: 訊息內容（bytes）
    - msg.qos: QoS 等級
    """
    try:
        # 解碼訊息（假設是 UTF-8 編碼）
        message = msg.payload.decode('utf-8')
        
        print(f"\n{'='*60}")
        print(f"📨 收到新訊息")
        print(f"主題: {msg.topic}")
        print(f"QoS: {msg.qos}")
        print(f"訊息內容: {message}")
        
        # 嘗試解析 JSON 格式
        try:
            json_data = json.loads(message)
            print(f"\n📊 JSON 資料解析:")
            for key, value in json_data.items():
                print(f"  - {key}: {value}")
        except json.JSONDecodeError:
            # 如果不是 JSON 格式，就當作普通文字處理
            pass
            
        print(f"{'='*60}\n")
        
    except Exception as e:
        print(f"✗ 處理訊息時發生錯誤: {e}")

# 斷線回調函數
def on_disconnect(client, userdata, rc):
    """當客戶端斷線時的回調函數"""
    if rc != 0:
        print(f"✗ 意外斷線，錯誤碼: {rc}")
    else:
        print("✓ 正常斷開連接")

print("✓ 回調函數定義完成")


✓ 回調函數定義完成


## 建立 MQTT 客戶端並連接


In [13]:
# 創建 MQTT 客戶端
client = mqtt.Client(client_id=MQTT_CLIENT_ID)

# 設置回調函數
client.on_connect = on_connect
client.on_subscribe = on_subscribe
client.on_message = on_message
client.on_disconnect = on_disconnect

# 連接到 MQTT Broker
print(f"正在連接到 MQTT Broker: {MQTT_BROKER}:{MQTT_PORT}...")
try:
    client.connect(MQTT_BROKER, MQTT_PORT, keepalive=60)
    print("✓ 連接請求已發送")
except Exception as e:
    print(f"✗ 連接時發生錯誤: {e}")


正在連接到 MQTT Broker: localhost:1883...
✓ 連接請求已發送


/tmp/ipykernel_35838/3082409398.py:2: DeprecationWarning: Callback API version 1 is deprecated, update to latest version
  client = mqtt.Client(client_id=MQTT_CLIENT_ID)


## 開始監聽訊息

執行此 cell 後，程式會持續監聽來自 MQTT Broker 的訊息。

**注意：** 這個 cell 會持續運行，直到您中斷執行或手動停止。


In [18]:
# 啟動網路循環（持續監聽訊息）
client.loop_start()

print("✓ MQTT 訂閱服務已啟動")
print("等待接收訊息中...")
print("(若要停止，請中斷執行此 cell)\n")

# 持續運行，等待接收訊息
# 在 Jupyter Notebook 中，可以讓它持續運行
# 或者設置一個時間限制來測試

try:
    # 等待 30 秒來接收訊息（可以根據需要調整時間）
    # 如果您想持續監聽，可以將下面的 sleep 時間改長，或者移除 try-except 讓它持續運行
    time.sleep(300)
    print("\n⏰ 30 秒監聽時間已到")
except KeyboardInterrupt:
    print("\n⚠️ 使用者中斷執行")
finally:
    print("正在停止訂閱服務...")
    client.loop_stop()
    client.disconnect()
    print("✓ 已停止並斷開連接")


✓ MQTT 訂閱服務已啟動
等待接收訊息中...
(若要停止，請中斷執行此 cell)


⚠️ 使用者中斷執行
正在停止訂閱服務...
✓ 已停止並斷開連接


## 持續監聽模式（可選）

如果您想要持續監聽訊息而不設時間限制，可以使用以下程式碼：

**執行前請注意：**
- 此模式會持續運行，直到您手動中斷
- 建議先執行上面的測試 cell，確認一切正常後再使用此模式


In [15]:
# 持續監聽模式
# 取消下面的註解來啟用持續監聽（取消註解後，程式會持續運行直到中斷）

"""
# 確保客戶端已連接
if not client.is_connected():
    client.connect(MQTT_BROKER, MQTT_PORT, keepalive=60)
    client.loop_start()
    time.sleep(1)

print("✓ 持續監聽模式已啟動")
print("等待接收訊息中...")
print("(按 Ctrl+C 或中斷 cell 來停止)\n")

try:
    # 持續運行
    while True:
        time.sleep(1)
except KeyboardInterrupt:
    print("\n⚠️ 使用者中斷執行")
finally:
    client.loop_stop()
    client.disconnect()
    print("✓ 已停止並斷開連接")
"""

print("(此 cell 目前未啟用，如需使用請取消註解)")


(此 cell 目前未啟用，如需使用請取消註解)


## 使用說明

1. **執行所有 cell**：按照順序執行上面的所有 cell

2. **測試訂閱**：
   - 執行到「開始監聽訊息」的 cell 後，程式會開始監聽
   - 在另一個終端或 Notebook 中執行 `mqtt_publish.py` 來發送測試訊息
   - 或者使用命令列工具：`mosquitto_pub -h localhost -t "客廳/溫度" -m "測試訊息"`

3. **停止監聽**：中斷執行監聽的 cell（點擊停止按鈕或按 Ctrl+C）

4. **查看訊息**：收到的訊息會顯示在上方，包括主題、QoS 和訊息內容
